# Fairness vs Efficiency: Balancing Human Factors with Optimization

This notebook demonstrates the **tradeoff between fairness and efficiency** in scheduling.

Understanding fairness-efficiency tradeoffs is important because:
- Perfect efficiency might create unfair schedules
- Perfect fairness might not be the most efficient approach
- Schedules must balance both concerns to succeed in practice
- Human factors affect schedule success, not just mathematical optimality


## Key Concepts

**Fairness** can mean different things:
- Equal hours: everyone works the same amount
- Equal distribution: undesirable work is shared equally
- Equal opportunities: everyone has same chance for preferred assignments

**Efficiency** means:
- Optimal resource utilization
- Minimizing cost
- Maximizing productivity

**The Tradeoff**:
- Efficient schedules might assign more work to more productive people (unfair)
- Fair schedules might distribute work equally regardless of productivity (inefficient)
- The challenge is finding an acceptable balance

**Critical insight**: Fairness and efficiency often conflict. The right balance depends on your priorities and context.


## Scenario: Staff Scheduling with Productivity Differences

You must schedule 5 employees with different productivity levels. You want to balance fairness (equal hours) with efficiency (maximize total output).

**Employees**:
- Alice: 10 units/hour (most productive)
- Bob: 8 units/hour
- Carol: 7 units/hour
- David: 6 units/hour
- Eve: 5 units/hour (least productive)

**Challenge**: How do you balance fairness with efficiency?


## Step 1: Install Required Packages (Colab)


In [1]:
%pip install matplotlib pandas numpy -q



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


## Step 2: Import Libraries


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## Step 3: Compare Fairness vs Efficiency Scenarios


In [3]:
# Employee productivity
employees = {
    'Alice': 10,
    'Bob': 8,
    'Carol': 7,
    'David': 6,
    'Eve': 5
}

total_hours_needed = 200  # Total hours of work needed

# Scenario 1: Maximum Efficiency (assign all work to most productive)
efficiency_hours = {'Alice': total_hours_needed, 'Bob': 0, 'Carol': 0, 'David': 0, 'Eve': 0}
efficiency_output = sum(employees[emp] * hours for emp, hours in efficiency_hours.items())

# Scenario 2: Maximum Fairness (equal hours for everyone)
fair_hours_per_person = total_hours_needed / len(employees)
fairness_hours = {emp: fair_hours_per_person for emp in employees.keys()}
fairness_output = sum(employees[emp] * hours for emp, hours in fairness_hours.items())

# Scenario 3: Balanced (weighted by productivity but with fairness constraint)
# Assign more hours to productive people, but limit the difference
balanced_hours = {}
productivity_sum = sum(employees.values())
for emp, prod in employees.items():
    # Weight by productivity but cap maximum hours
    base_hours = (prod / productivity_sum) * total_hours_needed
    max_hours = fair_hours_per_person * 1.5  # Cap at 1.5x average
    balanced_hours[emp] = min(base_hours, max_hours)

# Normalize to total
total_balanced = sum(balanced_hours.values())
balanced_hours = {emp: (hours / total_balanced) * total_hours_needed 
                  for emp, hours in balanced_hours.items()}
balanced_output = sum(employees[emp] * hours for emp, hours in balanced_hours.items())

# Calculate fairness metrics (coefficient of variation of hours)
def calc_fairness(hours_dict):
    hours_list = list(hours_dict.values())
    return np.std(hours_list) / np.mean(hours_list) if np.mean(hours_list) > 0 else 0

scenarios = pd.DataFrame({
    'Scenario': ['Maximum Efficiency', 'Maximum Fairness', 'Balanced'],
    'Total Output': [efficiency_output, fairness_output, balanced_output],
    'Fairness (CV)': [calc_fairness(efficiency_hours), 
                     calc_fairness(fairness_hours),
                     calc_fairness(balanced_hours)]
})

print("FAIRNESS VS EFFICIENCY COMPARISON:")
print("=" * 70)
display(scenarios.round(2))

print("\nKey Insights:")
print("  - Maximum Efficiency: Highest output, but very unfair (all work to Alice)")
print("  - Maximum Fairness: Most fair, but lower output (ignores productivity)")
print("  - Balanced: Compromise between fairness and efficiency")
print("\n  → The 'best' schedule depends on your priorities!")


FAIRNESS VS EFFICIENCY COMPARISON:


,Scenario,Total Output,Fairness (CV)
0,Maximum Efficiency,2000.00,2.00
1,Maximum Fairness,1440.00,0.00
2,Balanced,1522.22,0.24



Key Insights:
  - Maximum Efficiency: Highest output, but very unfair (all work to Alice)
  - Maximum Fairness: Most fair, but lower output (ignores productivity)
  - Balanced: Compromise between fairness and efficiency

  → The 'best' schedule depends on your priorities!
